In [1]:
__nbid__ = '0050'
__author__ = 'Mike Fitzpatrick <mike.fitzpatrick@noirlab.edu>, Alice Jacques <alice.jacques@noirlab.edu>, Astro Data Lab Team <datalab@noirlab.edu>'
__version__ = '20240607' # yyyymmdd
__keywords__ = ['query', 'vospace', 'mydb']

# How to use the Data Lab *Query Client* Service

### Table of contents
* [Summary](#summary)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#imports)
* [Review: General template for a simple query in SQL](#review)
* [Quick Example Query](#query)
* [Save to virtual storage VOSpace](#savevospace)
* [Save to remote database MyDB](#savemydb)
* [Import data into a MyDB table with QueryClient](#importmydb)
* [An asynchronous query example](#async)
* [Using profiles](#profiles)


<a class="anchor" id="summary"></a>
# Summary

This notebook documents how to query the Data Lab via the query client service. For full documentation see the <a href="https://datalab.noirlab.edu/docs/api/dl.html#module-dl.queryClient">API documentation</a>.


### The *queryClient* class


All queries are executed through the *queryClient.query()* method of the *queryClient* class. This takes as arguments:

| Argument | Description | Optional | Default  value | Supported Values |
|----------|-------------|----------|----------------|----------------|
| adql | The query to be submitted to the TAP service | No | None | string |
| sql | The query to be submitted to the DB directly | Yes | None | string |
| fmt | The requested format (if any) | Yes | ascii | ascii,csv,fits,hdf5,votable |
| out | The saved location (if any) | Yes | None | local filename, *vos://filename*, *mydb://tablename* |
| async_ | Indicates if the query is asynchronous | Yes | False | True/False |


All arguments are optional, except that one of *adql* or *sql* must be supplied.  The distinction between these two parameters is in how the *QueryClient* executes the query:  If *adql* is provided the query is sent to the TAP (Table Access Protocol) service, if *sql* is provided the query is sent directly to the database. The choice of execution depends on whether the query string contains ADQL-specific functions, or SQL constructs or DB extensions not understood by the TAP service.  For large queries there can also be a performance difference depending on the where/how the results are saved. 

#### Output formats
The results can be returned as whitespace delimited (*ascii*), CSV (*csv*), FITS object (*fits*), HDF5 (*hdf5*), or in VOTable format (*votable*). Note that if the results are saved to the user's database then the output format is ignored.

#### Saving results
If no save location is specified (no *out* param) then the results are returned directly. A save location beginning with the 'vos://' identifier indicates a location in the user's virtual storage to save the result. A save location beginning with the 'mydb://' identifier indicates the results are to be saved to a table in the user's remote database (MyDB). 

#### Asynchronous queries
Long queries should be run asynchronously and the service may refuse any synchronous query attempted if the projected query time is too long. A query can be submitted asynchronously by setting the *async_* parameter to *True*. A job id will then be returned.

    
The status of an asynchronous query can be checked with *queryClient.status(jobid)*. A return value of 'COMPLETED' indicates the query has terminated. A return value of 'ERROR' indicates that there was a problem with the query.    

The results of an asynchronous query (assuming that they were not saved to either the user's virtual storage or remote database) can be retrieved once the query has completed with *result = qc.query(adql=query,async_=True,wait=True,poll=1,verbose=1)*

### From Python code

The query client service can be called from Python code using the *datalab* module. This provides methods to access the various query client functions in the *QueryClient* subpackage. See the information [here](https://github.com/astro-datalab/datalab/blob/master/README.md).

Queries can be also run from the command line, e.g. on your local machine, using the datalab command line utility. Read about it in our GitHub repo [here](https://github.com/astro-datalab/datalab).


<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.


<a class="anchor" id="imports"></a>
# Imports and setup

This is the setup that is required to use the query client. The first thing to do is import the relevant Python modules.

In [2]:
# Standard lib
from getpass import getpass

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert

# Authentication
Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, un-comment the cell below and execute it:

In [3]:
#token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

'demo00'

<a class="anchor" id="review"></a>
# Review: General template for a simple query in SQL

### SQL is a way to describe to a database what you want from it
General template for a simple query written in SQL
```
SELECT something
FROM database.table
WHERE constraints
LIMIT 100
```
### Please see our intro notebook [JupyterPythonSQL101](https://github.com/astro-datalab/notebooks-latest/blob/master/01_GettingStartedWithDataLab/01_JupyterPythonSQL101.ipynb) for more info on this.

<a class="anchor" id="query"></a>
# A quick query

Let's say we want to fetch the g,r,i magnitudes from 10 objects in the SMASH DR1 data set, and retrieve the results as a CSV-formatted string:

In [4]:
query = 'SELECT gmag, rmag, imag FROM smash_dr1.object WHERE gmag<99 AND rmag<99 AND imag<99 LIMIT 10'
response = qc.query(sql = query, fmt = 'csv')
print (response)

gmag,rmag,imag
11.915434,11.797816,11.789193
11.937625,11.509244,11.325349
12.069924,11.693381,11.55774
12.074745,16.537102,17.018686
12.124366,11.802776,11.616948
12.135513,11.762072,11.546856
12.184068,11.844554,11.719124
12.210214,12.188986,12.318025
12.227514,11.763244,11.506152
12.249985,11.839742,11.703437



<a class="anchor" id="savevospace"></a>
# Saving results to virtual storage VOSpace

VOSpace is a convenient storage space for users to save their work. It can store any data or file type.  

Now we want to save the results from the same query to our virtual storage space instead.  By putting the query in a try-block we are able to trap errors when executing the query.  Note that running this cell multiple times will trigger an error and we use the Storage Manager Client to remove the file once we are done.

In [5]:
try:
    response = qc.query (sql=query, fmt='csv', out='vos://examplemags.csv')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)        
    else:
        print ("OK")

OK


Let's ensure the file was created in VOSpace:

In [6]:
sc.ls(name='vos://examplemags.csv')

'examplemags.csv'

Now let's remove the file we just created:

In [7]:
sc.rm (name='vos://examplemags.csv')

'OK'

Let's ensure the file was removed from VOSpace:

In [8]:
sc.rm (name='vos://examplemags.csv')

'A Node does not exist with the requested URI.'

<a class="anchor" id="savemydb"></a>
# Saving results to remote database MyDB
MyDB is a useful OS remote per-user relational database that can store data tables. Furthermore, the results of queries can be directly saved to MyDB, as we show in the following example:

In [9]:
try:
    response = qc.query (sql=query, fmt='csv', out='mydb://examplemags')
except Exception as e:
    # Handle any errors in the query.  By running this cell multiple times with the same
    # output file, or by using a bogus SQL statement, you can view various error messages.
    print (e.message)
else:
    if response is not None: 
        print (response)         
    else:
        print ("OK")

OK


Ensure the table has been saved to MyDB by calling the *mydb_list()* function:

In [10]:
print(qc.mydb_list(),"\n")

aaa
aab
basic_result
bgsfaint_dlnotebook
cmtestoutput
cmxmatchtest
desi_tile
df_xmatch
examplemags
fastspec_everest_z_lt_0p6
fiberassign
gaia_rc
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat
testcm
testingx
testresult2
testx2
testxmatchqueryout
tile
twomass_gaia1
twomass_pt1
twomasspsc
usno_objects
xmatchasyncout
xmatchasyncout2
 



Now let's drop the table from our MyDB.

In [11]:
qc.mydb_drop('examplemags')

'OK'

Ensure it has been removed by calling the *mydb_list()* function again:

In [12]:
print(qc.mydb_list(),"\n")

aaa
aab
basic_result
bgsfaint_dlnotebook
cmtestoutput
cmxmatchtest
desi_tile
df_xmatch
fastspec_everest_z_lt_0p6
fiberassign
gaia_rc
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat
testcm
testingx
testresult2
testx2
testxmatchqueryout
tile
twomass_gaia1
twomass_pt1
twomasspsc
usno_objects
xmatchasyncout
xmatchasyncout2
 



<a class="anchor" id="importmydb"></a>
# Import data into a MyDB table with QueryClient

Users can use the *mydb_import* function to import data saved on a local computer or import data from VOSpace into a MyDB data table. The data must be in the form of either a CSV file or Pandas Dataframe object in order to load it into MyDB.

First let's query some data from the Data Lab database and save it locally as a CSV file:

In [13]:
query = "select * from gaia_dr1.gaia_source limit 10"
qc.query (sql=query, fmt='csv', out='./gaiaresult.csv')

'OK'

Next we will use the *mydb_import* function to import the locally stored CSV file into a MyDB table:

In [14]:
qc.mydb_import('testresult','./gaiaresult.csv')

'OK'

Let's ensure it's there by calling the *mydb_list()* function:

In [15]:
print(qc.mydb_list(),"\n")

aaa
aab
basic_result
bgsfaint_dlnotebook
cmtestoutput
cmxmatchtest
desi_tile
df_xmatch
fastspec_everest_z_lt_0p6
fiberassign
gaia_rc
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat
testcm
testingx
testresult
testresult2
testx2
testxmatchqueryout
tile
twomass_gaia1
twomass_pt1
twomasspsc
usno_objects
xmatchasyncout
xmatchasyncout2
 



Similarly, we can use the *mydb_import* function to import data from VOSpace into a MyDB table:

In [16]:
qc.mydb_import('testresult2','vos://newmags.csv')

'OK'

In [17]:
print(qc.mydb_list(),"\n")

aaa
aab
basic_result
bgsfaint_dlnotebook
cmtestoutput
cmxmatchtest
desi_tile
df_xmatch
fastspec_everest_z_lt_0p6
fiberassign
gaia_rc
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat
testcm
testingx
testresult
testresult2
testx2
testxmatchqueryout
tile
twomass_gaia1
twomass_pt1
twomasspsc
usno_objects
xmatchasyncout
xmatchasyncout2
 



Finally, for clean-up purposes, let's remove the two tables we just imported into MyDB by using the *mydb_drop* function:

In [18]:
qc.mydb_drop('testresult')
qc.mydb_drop('testresult2')

'relation "testresult2" not known'

And let's make sure the two tables were removed from MyDB by using the *mydb_list()* function:

In [19]:
print(qc.mydb_list(),"\n")

aaa
aab
basic_result
bgsfaint_dlnotebook
cmtestoutput
cmxmatchtest
desi_tile
df_xmatch
fastspec_everest_z_lt_0p6
fiberassign
gaia_rc
gaia_sample
gaia_sample_xmatch
gals
lowmassagn_dlnotebook
secondary_dark_subset
sv1targets_bright_secondary
sv1targets_dark_secondary
tbl_stat
testcm
testingx
testresult2
testx2
testxmatchqueryout
tile
twomass_gaia1
twomass_pt1
twomasspsc
usno_objects
xmatchasyncout
xmatchasyncout2
 



<a class="anchor" id="async"></a>
# An asynchronous query

We now want to run a longer query, say, counting the total number of objects in USNO-B1, and need to do it asynchronously. To do this we will submit the query as normal but with the *async_* argument indicated.

In [20]:
query = 'SELECT count(*) FROM usno.b1'

In [21]:
result = qc.query(sql=query,async_=True,wait=True,poll=15,verbose=1)

EXECUTING
Status = EXECUTING; elapsed time: 15, timeout in 285
EXECUTING
Status = EXECUTING; elapsed time: 30, timeout in 270
EXECUTING
Status = EXECUTING; elapsed time: 45, timeout in 255
EXECUTING
Status = COMPLETED; elapsed time: 60, timeout in 240
Retrieving results


In [22]:
print(result)

COUNT
1045175762



### Please see our [JupyterPythonSQL101](https://github.com/astro-datalab/notebooks-latest/blob/master/01_GettingStartedWithDataLab/01_JupyterPythonSQL101.ipynb) notebook for more information on getting jobid, checking status until 'COMPLETED', and retrieving results with qc.results(). 

<a class="anchor" id="profiles"></a>
# Using profiles

Sometimes, different datasets (or versions of the same dataset) may reside on different backend servers and a user may want to work explicitly with a (typically older) dataset. In some cases these servers will be used only by developers or those with restricted access. Also, external TAP services are accessible from within Data Lab. In both use cases, the *qc.profiles()* function comes in handy.

The first thing to do is see what profiles are available:

In [23]:
profilelist = qc.list_profiles()
print (profilelist)

       GALEX-DR6   GALEX DR6 TAP service (29 Tables, TAP Only)
            GAVO   GAVO Data Center TAP service (149 Tables, TAP Only)
         HEASARC   HEASARC Xamin TAP Service (921 Tables, TAP Only)
            IRSA   IRSA TAP Service (478 Tables, TAP Only)
        SDSS-DR9   SDSS DR9 TAP service (92 Tables, TAP Only)
          SIMBAD   SIMBAD TAP service (47 Tables, TAP Only)
    STScI-RegTAP   STScI Registry TAP service (18 Tables, TAP Only)
          Vizier   TAP Vizier query engine (34381 Tables, TAP Only)



The thing to note in the output here are names such as '*GAVO*', '*SDSS-DR9*', etc;  **These profiles refer to external TAP services that can be accessed using the Query Manager interface.** Only a few are configured at the moment as we work on ways to automatically discover the >100 such services and provide useful listings of what they contain.

Let's see how we can query one and save the result to our Virtual Storage VOSpace:

In [24]:
qc.set_profile('GAVO')
query = 'SELECT top 10 * FROM sdssdr7.sources'
response = qc.query(adql = query, fmt = 'csv', out="vos://gavo_out.csv")
print (response)

OK


In this case we queried an SDSS DR7 table at the TAP service run by GAVO in Heidelberg. 

Let's ensure it has been saved to VOSpace:

In [25]:
listing = sc.ls (name='vos://',format='long')
print (listing)

-rw-rw-r-x  demo00  4194304  27 Dec 2021 18:32  100000results.csv
-rw-rw-r-x  demo00  14159658  27 Dec 2021 18:32  100000x12results.csv
-rw-rw-r-x  demo00  4194304  27 Dec 2021 18:32  100000x5results.csv
-rw-rw-r-x  demo00      34  20 Jul 2022 14:12  235212.85-012029.6.txt
drwxrwxr-x  demo00       0  27 Dec 2021 18:32  HIPASS_images_most_massive/
drwxrwxr-x  demo00       0  27 Dec 2021 18:32  HIPASS_spectra_most_massive/
drwxrwxr-x  demo00       0  04 Apr 2023 14:15  PhotometryTables/
-rw-rw-r-x  demo00   35733  27 Dec 2021 18:32  a2_small.csv
-rw-rw-r-x  demo00     202  27 Dec 2021 18:32  canaryfiletwo.csv
-rw-rw-r-x  demo00  7878344  27 Dec 2021 18:32  cutout.fits
-rw-rw-r-x  demo00     489  27 Dec 2021 18:32  cutout2.fits
drwxrwxr-x  demo00       0  27 Dec 2021 18:32  directory1/
-rw-rw-r-x  demo00    7327  27 Dec 2021 18:32  fooa.csv
-rw-rw-r-x  demo00  7936244  16 Mar 2023 14:34  gaia100pc.csv
-rw-rw-r-x  demo00  178620  19 Apr 2023 14:41  gaia_hvs.csv
-rw-rw-r-x  demo00  230767  

We can load the data set and e.g. convert to a Pandas data frame, like this:

In [26]:
data = sc.get(fr = 'vos://gavo_out.csv', to = '')
df = convert(data)
df

,objid,run,rerun,camcol,fieldid,obj,ra,dec,raerr,decerr,...,offsetra_i,offsetdec_i,i,err_i,epoch_i,offsetra_z,offsetdec_z,z,err_z,epoch_z
0,758883089249337439,6182,648,1,758883089249337344,95,331.786804,41.387806,0.000117,0.000113,...,-0.000064,0.000065,19.9785,0.092936,2006.41,4.666670e-07,-2.388890e-07,22.8831,2.086310,2006.41
1,758883089249337433,6182,648,1,758883089249337344,89,331.780687,41.389283,0.000035,0.000030,...,-0.000111,-0.000117,20.7977,0.097531,2006.41,-1.363640e-04,3.201940e-05,20.1367,0.202043,2006.41
2,758883089249337434,6182,648,1,758883089249337344,90,331.784089,41.389143,0.000033,0.000039,...,-0.000402,0.000139,21.4730,0.153602,2006.41,-3.638890e-07,3.530560e-06,20.9629,0.351456,2006.41
3,758883089249337438,6182,648,1,758883089249337344,94,331.793057,41.390098,0.000036,0.000035,...,0.000087,0.000052,24.3450,1.940740,2006.41,8.762780e-05,5.144440e-05,22.5182,1.635630,2006.41
4,758883089249339591,6182,648,1,758883089249337344,2247,331.801543,41.390926,0.000016,0.000015,...,0.000001,0.000011,20.6834,0.059923,2006.41,-5.135830e-05,-6.525000e-06,20.2915,0.163408,2006.41
5,758883089249337441,6182,648,1,758883089249337344,97,331.797318,41.390873,0.000070,0.000068,...,-0.000085,-0.000045,20.5697,0.132147,2006.41,5.000000e-07,-1.944440e-07,24.7128,0.622408,2006.41
6,758883089249337437,6182,648,1,758883089249337344,93,331.798537,41.392072,0.000033,0.000032,...,0.000005,-0.000006,20.1749,0.120331,2006.41,-4.353060e-05,3.612140e-04,24.0276,1.432990,2006.41
7,758883089249337419,6182,648,1,758883089249337344,75,331.792733,41.392017,0.000014,0.000013,...,-0.000004,-0.000005,15.8837,0.003775,2006.41,-4.591670e-06,-1.697220e-06,15.5639,0.005426,2006.41
8,758883089249337423,6182,648,1,758883089249337344,79,331.796938,41.394596,0.000015,0.000014,...,-0.000003,-0.000026,19.8373,0.032767,2006.41,-6.108330e-05,-6.421390e-05,19.3057,0.074711,2006.41
9,758883089249339570,6182,648,1,758883089249337344,2226,331.775376,41.392066,0.000018,0.000018,...,0.000031,0.000014,NaN,NaN,2006.41,-4.472220e-07,7.008330e-06,NaN,NaN,2006.41


We can get the details of a particular profile by including the name of the profile as an argument in the *list_profiles* method:

In [27]:
qc.list_profiles('default')

{'description': 'default-db03',
 'type': 'hidden',
 'database': 'tapdb',
 'mydb_user': 'datalab',
 'accessURL': 'http://gp01.datalab.noirlab.edu:8080/ivoa-dal/tap',
 'vosRoot': 'vos://datalab.noirlab!vospace',
 'tempfilePath': '/net/dl2/temp',
 'resultStorePath': '/net/dl1/tap_data/resultStoreImpl'}

So let's try a query against the default profile - let's get a list of all tables in the default database.  Note that in this case we are accessing the *information_schema* table of the database, this table is not included in the TAP service and so we **must** use the *sql* argument to talk directly to the database.

In [28]:
sql = 'SELECT table_catalog, table_schema, table_name FROM information_schema.tables'
try:
    qc.set_profile('default')
    default = qc.query(sql=sql)
except Exception as e:
    print (e.message)
else:
    print (default)

table_catalog,table_schema,table_name
tapdb,pg_catalog,pg_aggregate
tapdb,pg_catalog,pg_am
tapdb,pg_catalog,pg_amop
tapdb,pg_catalog,pg_amproc
tapdb,pg_catalog,pg_attrdef
tapdb,pg_catalog,pg_attribute
tapdb,pg_catalog,pg_auth_members
tapdb,pg_catalog,pg_authid
tapdb,pg_catalog,pg_available_extension_versions
tapdb,pg_catalog,pg_available_extensions
tapdb,pg_catalog,pg_cast
tapdb,pg_catalog,pg_class
tapdb,pg_catalog,pg_collation
tapdb,pg_catalog,pg_config
tapdb,pg_catalog,pg_constraint
tapdb,pg_catalog,pg_conversion
tapdb,pg_catalog,pg_cursors
tapdb,pg_catalog,pg_database
tapdb,pg_catalog,pg_db_role_setting
tapdb,pg_catalog,pg_default_acl
tapdb,pg_catalog,pg_depend
tapdb,pg_catalog,pg_description
tapdb,pg_catalog,pg_enum
tapdb,pg_catalog,pg_event_trigger
tapdb,pg_catalog,pg_extension
tapdb,pg_catalog,pg_file_settings
tapdb,pg_catalog,pg_foreign_data_wrapper
tapdb,pg_catalog,pg_foreign_server
tapdb,pg_catalog,pg_foreign_table
tapdb,pg_catalog,pg_group
tapdb,pg_catalog,pg_hba_file_rules
t